![The Pokémon Logo](pokemon-logo.png)

# **When Best to Buy to Collect 'em All**
When I was a kid, [Pokémon](https://en.wikipedia.org/wiki/Pok%C3%A9mon) was all the rage.  At 9, I was regularly watching a VHS tape (dating myself, I know) that simply *talked* about the idea of catching them all.  I was hooked.  Like every fan's relationship with their hobby or interest, I had my ups and downs with the franchise and its various media, yet my adoration persisted.  There was one line of products that really caught my attention as a child.  It was the [Trading Card Game](https://en.wikipedia.org/wiki/Pok%C3%A9mon_Trading_Card_Game) (TCG, for short), and came shortly after the [television series](https://en.wikipedia.org/wiki/Pok%C3%A9mon_(TV_series)) and [games](https://en.wikipedia.org/wiki/List_of_Pok%C3%A9mon_video_games) and was just as big of a smash hit as the aforementioned two.  I jumped on the card game, doing everything from collect to 'battle' with them.  I collected in a large spurt (my mother's pocketbook suffered greatly) and was generally a giant pest.  I was spoiled, but unfortunately for me and fortunately for her, my love was short-lived.  Sadly, however, it was not because I just stopped, but it was more mental.

Throughout my life I've been battling depression.  In college, I was formally diagnosed with dysthymia which is just fancy doctor-speak for persistent, long-term depression.  In my teens and twenties it was at its worst, I all but fell out of love with the card game and a lot of things in life.  It was only in my thirties that I realized hobbies are an important part of one's life and sense of self.  So, during the pandemic, I jumped into my closet and found the old cards I'd collected.  Unfortunately, I didn't take good care of my cards.  I remain undeterred though.  I'm 34 now, and my desire has never been higher.

So, I'm on a quest to fulfill what my childhood self wanted.  This notebook and project are with that in mind, to find deals, and figure out through pricing history and trends how best I can approach what almost every Pokémon aficionado strives so hard to do: to collect 'em all.

#### **Goal(s) for this project**
- analyzing trends and the overall history of sealed product and singles (individual cards) to find the best time to purchase them
- people are different and like different types of Pokémon.  i want to analyze if those different people have different purchasing habits.
- for longer term cards, i.e. 'vintage', i'll look through the various sets release and see if they forever mature or their price dips as well
- numbers will be skewed as we were just coming off of the pandemic

#### **Expectations and Methodology** <img src="pikachu-magnifying-glass.png" width="325.33333333333333333333333333333" height="341.33333333333333333333333333333" align="right"/>
Insofar as my expectations are concerned, I think this particular project is going to be difficult.  I'm doing a bunch of different things than my prior cases in Jupyter Notebook and this is a lot more invovled, mechanically, than most of my old stuff.

First things first: this project wouldn't be possible without an API or a webscraper.  After a pretty laborious search through the muck of Kaggle, Google, and premium APIs, I found a [single RESTful API](https://pokemontcg.io/), which is the easier architecture of the two available (the other being SOAP) to make use of.  At this moment, I'm going to mess with the API and see if it returns what I want.  What I expect from this, insofar as the 'spine' of this project is concerned, is threefold:

1. I want to take 'snapshots' of certain windows when it comes to pricing items.  For all of them, it'll be on the intervals of 1, 2, 3, 6, 9, and a year.  Then, provided the item has been out long enough, I'll also be looking at a 1 and a half, 2, and 3 years, maybe more.  This will focus on sets that came out when the database that we're literating through actually existed.  This timeframe will be within the past 3 years.  Additionally, we'll also go through sets that were popular and weren't to see how the demand is structured and if it's the same.
2. There are, ultimately, favorite Pokémon in the fandom.  People like Pikachu more than Spearow, or Charizard more than Feraligatr.  So, I want to see if pricing trends differ based on popularity.
3. Similarly, I will also look at the trends of singles and sets and the relation.

I will then make conclusions based off of what I've learned.

##### *Unachieved goals*:
* I want to get another financial RESTful API, probably eBay?
* Document and rehearse what I'm doing so to better understand.
* Provide, bare minimum, 6 graphs and tables, 2 of which are made in Tableau.
* Sets are not priced according to the sole API I've pulled.

Pseudocode (use functions):
assign main variables

manipulate dates so that we have our time frame
convert 'release date' (string) from API calls into date time
calculate intervals (1, 2, 3, 6, 9, 12 months (a year) 1 1/2, 2, 3, and - maybe more - years )
filter and fix data based on dates
    maybe throw these into a dictionary?  maybe consider making a .csv of these and then creating various viz's


In [ ]:
# importing various modules
import requests as rq
import datetime as date
import pandas as pd
import matplotlib as mpl
import seaborn as sb

# instanting add and header for our API key
url = "https://api.pokemontcg.io/v2/sets"

headers = {
    "X-Api-Key": "025036c3-abbf-4f6b-929e-f537e499759c"
    }

# creating a test response
response = rq.get(url, headers=headers)

# if statement to produce error response if faulty
if response.status_code == 200:
    data = response.json()  # assigning json format to variable
    print(data)             # printing said variable
else:
    print(f"Error: {response.status_code}")